# Deep Learning 基礎講座　最終課題: 脳波分類

## 概要
被験者が画像を見ているときの脳波から，その画像がどのカテゴリに属するかを分類するタスク．
- サンプル数: 訓練 118,800 サンプル，検証 59,400 サンプル，テスト 59,400 サンプル
- クラス数: 5
- 入力: 脳波データ（チャンネル数 x 系列長）
- 出力: 対応する画像のクラス
- 評価指標: Top-1 accuracy

### 元データセット ([Gifford2022 EEG dataset](https://osf.io/3jk45/)) との違い

- 本コンペでは難易度調整の目的で元データセットにいくつかの改変を加えています．

1. 訓練セットのみの使用
  - 元データセットでは訓練データに存在しなかったクラスの画像を見ているときの脳波においてテストが行われますが，これは難易度が非常に高くなります．
  - 本コンペでは元データセットの訓練セットを再分割し，訓練時に存在した画像に対応する別の脳波において検証・テストを行います．

2. クラス数の減少
  - 元データセット（の訓練セット）では16,540枚の画像に対し，1,654のクラスが存在します．
    - e.g. `aardvark`, `alligator`, `almond`, ...
  - 本コンペでは1,654のクラスを，`animal`, `food`, `clothing`, `tool`, `vehicle`の5つにまとめています．
    - e.g. `aardvark -> animal`, `alligator -> animal`, `almond -> food`, ...

### 考えられる工夫の例

- 音声モデルの導入
  - 脳波と同じ波である音声を扱うアーキテクチャを用いることが有効であると知られています．
  - 例）Conformer [[Gulati+ 2020](https://arxiv.org/abs/2005.08100)]
- 画像データを用いた事前学習
  - 本コンペのタスクは脳波のクラス分類ですが，配布してある画像データを脳波エンコーダの事前学習に用いることを許可します．
  - 例）CLIP [Radford+ 2021]
  - 画像を用いる場合は[こちら](https://osf.io/download/3v527/)からダウンロードしてください．
- 過学習を防ぐ正則化やドロップアウト


## 修了要件を満たす条件
- ベースラインモデルのbest test accuracyは38.8%となります．**これを超えた提出のみ，修了要件として認めます**．
- ベースラインから改善を加えることで，55%までは性能向上することを運営で確認しています．こちらを 1 つの指標として取り組んでみてください．

## 注意点
- 最終的な予測モデルは，**配布している訓練データを用いて学習**（ファインチューニング含む）したものとしてください．
- 学習を行わず，**事前学習済みモデルの知識のみを利用した推論は禁止**します．  
（例: ChatGPT 等の LLM に入力して推論を得るのみ）

### 事前学習モデルの利用
許可される事項
- **構成要素としての事前学習モデルの利用**: 自身で実装したアーキテクチャの一部（特徴抽出，埋め込みなど）として事前学習モデル（BERT，ViT など）を利用することは可能です．
- **ファインチューニング**: 上記の用途で利用している事前学習モデルのファインチューニングは可能です．

禁止される事項  
- **タスク解決用の事前学習モデルの利用**: transformers などで提供されている，対象タスクを直接解くための事前学習モデルでそのまま推論のみ，またはファインチューニングのみで利用することは禁止とします．
  - 禁止事項の例: VQA タスクを直接解くための事前学習モデルを VQA タスクで利用する．

## 1.準備

In [9]:
# omnicampus 実行用
!pip install ipywidgets

In [10]:
# ライブラリのインポートとシード固定
import os, sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from einops.layers.torch import Rearrange
from einops import repeat
from glob import glob
from termcolor import cprint
from tqdm.notebook import tqdm

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## 2.データセット

ノートブックと同じディレクトリに`data/`が存在することを確認してください．

In [11]:
class ThingsEEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str) -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.num_classes = 5
        self.num_subjects = 10

        self.X = np.load(f"data/{split}/eeg.npy")
        self.X = torch.from_numpy(self.X).to(torch.float32)
        self.subject_idxs = np.load(f"data/{split}/subject_idxs.npy")
        self.subject_idxs = torch.from_numpy(self.subject_idxs)

        if split in ["train", "val"]:
            self.y = np.load(f"data/{split}/labels.npy")
            self.y = torch.from_numpy(self.y)

        print(f"EEG: {self.X.shape}, labels: {self.y.shape if hasattr(self, 'y') else None}, subject indices: {self.subject_idxs.shape}")

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

## 3.ベースラインモデル

In [12]:
class ConvBlock(nn.Module):
    def __init__(
        self,
        in_dim,
        out_dim,
        kernel_size: int = 3,
        p_drop: float = 0.1,
    ) -> None:
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        self.conv0 = nn.Conv1d(in_dim, out_dim, kernel_size, padding="same")
        self.conv1 = nn.Conv1d(out_dim, out_dim, kernel_size, padding="same")
        # self.conv2 = nn.Conv1d(out_dim, out_dim, kernel_size) # , padding="same")

        self.batchnorm0 = nn.BatchNorm1d(num_features=out_dim)
        self.batchnorm1 = nn.BatchNorm1d(num_features=out_dim)

        self.dropout = nn.Dropout(p_drop)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        if self.in_dim == self.out_dim:
            X = self.conv0(X) + X  # skip connection
        else:
            X = self.conv0(X)

        X = F.gelu(self.batchnorm0(X))

        X = self.conv1(X) + X  # skip connection
        X = F.gelu(self.batchnorm1(X))

        # X = self.conv2(X)
        # X = F.glu(X, dim=-2)

        return self.dropout(X)


class BasicConvClassifier(nn.Module):
    def __init__(
        self,
        num_classes: int,
        seq_len: int,
        in_channels: int,
        hid_dim: int = 128
    ) -> None:
        super().__init__()

        self.blocks = nn.Sequential(
            ConvBlock(in_channels, hid_dim),
            ConvBlock(hid_dim, hid_dim),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Linear(hid_dim, num_classes),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """_summary_
        Args:
            X ( b, c, t ): _description_
        Returns:
            X ( b, num_classes ): _description_
        """
        X = self.blocks(X)

        return self.head(X)

## 4.訓練実行

In [14]:
%load_ext tensorboard
%tensorboard --logdir tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 30804), started 0:17:13 ago. (Use '!kill 30804' to kill it.)

In [17]:
# ハイパラ
lr = 0.001
batch_size = 512
epochs = 80

# ------------------
#    Dataloader
# ------------------
train_set = ThingsEEGDataset("train") # ThingsMEGDataset("train")
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
val_set = ThingsEEGDataset("val") # ThingsMEGDataset("val")
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=False
)

# ------------------
#       Model
# ------------------
model = BasicConvClassifier(
    train_set.num_classes, train_set.seq_len, train_set.num_channels
).to("cuda")

# ------------------
#     Optimizer
# ------------------
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# ------------------
#   Start training
# ------------------
max_val_acc = 0
def accuracy(y_pred, y):
    return (y_pred.argmax(dim=-1) == y).float().mean()

writer = SummaryWriter("tensorboard")

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    model.train()
    for X, y, subject_idxs in tqdm(train_loader, desc="Train"):
        X, y = X.to("cuda"), y.to("cuda")

        y_pred = model(X)

        loss = F.cross_entropy(y_pred, y)
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(y_pred, y)
        train_acc.append(acc.item())

    model.eval()
    for X, y, subject_idxs in tqdm(val_loader, desc="Validation"):
        X, y = X.to("cuda"), y.to("cuda")

        with torch.no_grad():
            y_pred = model(X)

        val_loss.append(F.cross_entropy(y_pred, y).item())
        val_acc.append(accuracy(y_pred, y).item())

    print(f"Epoch {epoch+1}/{epochs} | \
        train loss: {np.mean(train_loss):.3f} | \
        train acc: {np.mean(train_acc):.3f} | \
        val loss: {np.mean(val_loss):.3f} | \
        val acc: {np.mean(val_acc):.3f}")

    writer.add_scalar("train_loss", np.mean(train_loss), epoch)
    writer.add_scalar("train_acc", np.mean(train_acc), epoch)
    writer.add_scalar("val_loss", np.mean(val_loss), epoch)
    writer.add_scalar("val_acc", np.mean(val_acc), epoch)

    torch.save(model.state_dict(), "model_last.pt")

    if np.mean(val_acc) > max_val_acc:
        cprint("New best. Saving the model.", "cyan")
        torch.save(model.state_dict(), "model_best.pt")
        max_val_acc = np.mean(val_acc)

EEG: torch.Size([59400, 17, 100]), labels: torch.Size([118800]), subject indices: torch.Size([118800])
EEG: torch.Size([59400, 17, 100]), labels: torch.Size([59400]), subject indices: torch.Size([59400])
Epoch 1/80


Train:   0%|          | 0/117 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 1/80 |         train loss: 1.501 |         train acc: 0.375 |         val loss: 1.487 |         val acc: 0.389
New best. Saving the model.
Epoch 2/80


Train:   0%|          | 0/117 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 79/80 |         train loss: 1.009 |         train acc: 0.593 |         val loss: 2.132 |         val acc: 0.305
Epoch 80/80


Train:   0%|          | 0/117 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

Epoch 80/80 |         train loss: 1.015 |         train acc: 0.596 |         val loss: 2.146 |         val acc: 0.286


## 5.評価

In [15]:
# ------------------
#    Dataloader
# ------------------
test_set = ThingsEEGDataset("test")
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False
)

# ------------------
#       Model
# ------------------
model = BasicConvClassifier(
    test_set.num_classes, test_set.seq_len, test_set.num_channels
).to("cuda")
model.load_state_dict(torch.load("model_best.pt", map_location="cuda"))

# ------------------
#  Start evaluation
# ------------------
preds = []
model.eval()
for X, subject_idxs in tqdm(test_loader, desc="Evaluation"):
    preds.append(model(X.to("cuda")).detach().cpu())

preds = torch.cat(preds, dim=0).numpy()
np.save("submission", preds)
print(f"Submission {preds.shape} saved.")

EEG: torch.Size([59400, 17, 100]), labels: None, subject indices: torch.Size([59400])


Evaluation:   0%|          | 0/117 [00:00<?, ?it/s]

Submission (59400, 5) saved.


## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (EEG)」から提出してください．

- `submission.npy`
- `model_last.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook

In [16]:
from zipfile import ZipFile

model_path = "model_best.pt"
notebook_path = "DLBasics2025_competition_EEG_baseline.ipynb"

with ZipFile("submission.zip", "w") as zf:
    zf.write("submission.npy")
    zf.write(model_path)
    zf.write(notebook_path)